# Text Processing

This notebook imports scraped data, splits text column to words, extracts and normalizes tokens. Next, tokens are converted to embeddings.

In [44]:
### Setup

# pip install spacy
# python -m spacy download de_core_news_sm  - for small model (13 MB)
# python -m spacy download de_core_news_md  - for medium model (42 MB)
# source - https://spacy.io/models/de


import spacy
from spacy.lang.de.examples import sentences 


# Example from documentation
nlp = spacy.load("de_core_news_sm")
doc = nlp(sentences[0])
print(doc.text)
for token in doc:
    print(token.text, token.pos_, token.dep_)

Die ganze Stadt ist ein Startup: Shenzhen ist das Silicon Valley für Hardware-Firmen
Die DET nk
ganze ADJ nk
Stadt NOUN sb
ist AUX ROOT
ein DET nk
Startup NOUN pd
: PUNCT punct
Shenzhen NOUN sb
ist AUX cj
das DET nk
Silicon PROPN pnc
Valley PROPN sb
für ADP mnr
Hardware-Firmen NOUN nk


In [45]:
## Data loading

import pandas as pd

standard_data = pd.read_csv('./data/derstandard_frontpage_data.csv')[0:10000]
standard_data

,title,subtitle,link,datetime,kicker,n_posts,storylabels
0,Liverpool nach großem Zittern im Ligacup-Halbf...,Minamino rettete die Reds in der Nachspielzeit...,https://www.derstandard.at/story/2000132122809...,2021-12-22T23:47,Premier League,7,NaN
1,USA lassen Pfizers Covid-Tablette Paxlovid für...,Am Dienstag wurden in Österreich 2.269 Neuinfe...,https://www.derstandard.at/jetzt/livebericht/2...,2021-12-22T23:39,Omikron-Welle,22127,NachleseLivebericht
2,Real Madrid baut Vorsprung in LaLiga aus,Madrilenen nach 2:1 in Bilbao acht Punkte vor ...,https://www.derstandard.at/story/2000132122694...,2021-12-22T23:38,Primera Division,40,NaN
3,Paris Saint-Germain wendet zweite Liga-Niederl...,Icardi besorgteAusgleichstreffergegen Grbic-Te...,https://www.derstandard.at/story/2000132122616...,2021-12-22T23:30,Fußball,3,NaN
4,OSZE verkündet Einigung auf Waffenstillstand i...,"DieKonfliktparteiensollen zugestimmt haben, da...",https://www.derstandard.at/story/2000132122215...,2021-12-22T22:33,Weihnachtsfrieden,44,NaN
...,...,...,...,...,...,...,...
9995,Kocher kündigt Verlängerung der Corona-Sonderb...,Bei derSonderbetreuungszeitwerden die Lohnkost...,https://www.derstandard.at/story/2000134171664...,2022-03-16T18:55,Lohnkosten,8,NaN
9996,Ukraine-Krieg kostet Autobauer Rendite,Vor allem bei BMW hinterlässt die Ukraine-Kris...,https://www.derstandard.at/story/2000134170212...,2022-03-16T18:47,Geopolitische Probleme,15,NaN
9997,Flüchtlingsversorgung: Herausforderung annehmen,"Sollte es nicht zeitnah gelingen, die russisch...",https://www.derstandard.at/story/2000134170520...,2022-03-16T18:40,Irene Brickner,69,Kommentar
9998,Nach der Strabag geht auch Sigi Wolf auf Dista...,Die Kooperation von Steyr mit der russischen G...,https://www.derstandard.at/story/2000134170253...,2022-03-16T18:37,Kooperation,52,NaN


## Part 1,2: Tokenizing + lemmatizing text columns

First, title and subtitile columns are split into individual words. Second, words are replaced by their standard forms (großem - groß, rettete - retten). Finally, the list of standard forms is joined back to a string.

In [46]:
# Initializing two columns
standard_data['title_tokens'] = standard_data['title']
standard_data['subtitle_tokens'] = standard_data['subtitle']

# Load model
nlp = spacy.load("de_core_news_sm")
nlp.get_pipe('lemmatizer')


In [47]:
# loops through rows

for index, row in standard_data.iterrows():
    
    text = nlp(row['title_tokens'].replace("-", " ").replace(",", " ").replace(": ", " "))
    token_list = ' '.join([token.lemma_ for token in text]) # list of standardized tokens joined to a string
    standard_data.at[index, 'title_tokens'] = token_list # store in the dataframe

    # try except because subtitle is sometimes empty, then it outputs error

    try:

        text = nlp(row['subtitle_tokens'])
        token_list = ' '.join([token.lemma_ for token in text]) # list of standardized tokens joined to a string

    except:
        token_list = '' # if empty, token list empty

    standard_data.at[index, 'subtitle_tokens'] = token_list # store in the dataframe



In [48]:
standard_data

,title,subtitle,link,datetime,kicker,n_posts,storylabels,title_tokens,subtitle_tokens
0,Liverpool nach großem Zittern im Ligacup-Halbf...,Minamino rettete die Reds in der Nachspielzeit...,https://www.derstandard.at/story/2000132122809...,2021-12-22T23:47,Premier League,7,NaN,Liverpool nach groß Zittern in Ligacup Halbfinal,Minamino retten der Red in der Nachspielzeit i...
1,USA lassen Pfizers Covid-Tablette Paxlovid für...,Am Dienstag wurden in Österreich 2.269 Neuinfe...,https://www.derstandard.at/jetzt/livebericht/2...,2021-12-22T23:39,Omikron-Welle,22127,NachleseLivebericht,USA lassen Pfizer Covid Tablette Paxlovid für ...,an Dienstag werden in Österreich 2.269 Neuinfe...
2,Real Madrid baut Vorsprung in LaLiga aus,Madrilenen nach 2:1 in Bilbao acht Punkte vor ...,https://www.derstandard.at/story/2000132122694...,2021-12-22T23:38,Primera Division,40,NaN,Real Madrid bauen Vorsprung in LaLiga aus,Madrilen nach 2:1 in Bilbao acht Punkt vor FC ...
3,Paris Saint-Germain wendet zweite Liga-Niederl...,Icardi besorgteAusgleichstreffergegen Grbic-Te...,https://www.derstandard.at/story/2000132122616...,2021-12-22T23:30,Fußball,3,NaN,Paris Saint Germain wenden zweiter Liga Nieder...,Icardi besorgteAusgleichstreffergegen Grbic-Te...
4,OSZE verkündet Einigung auf Waffenstillstand i...,"DieKonfliktparteiensollen zugestimmt haben, da...",https://www.derstandard.at/story/2000132122215...,2021-12-22T22:33,Weihnachtsfrieden,44,NaN,OSZE verkünden Einigung auf Waffenstillstand i...,DieKonfliktparteiensollen zustimmen haben -- d...
...,...,...,...,...,...,...,...,...,...
9995,Kocher kündigt Verlängerung der Corona-Sonderb...,Bei derSonderbetreuungszeitwerden die Lohnkost...,https://www.derstandard.at/story/2000134171664...,2022-03-16T18:55,Lohnkosten,8,NaN,Kocher kündigen Verlängerung der Corona Sonder...,bei derSonderbetreuungszeitwerden der Lohnkost...
9996,Ukraine-Krieg kostet Autobauer Rendite,Vor allem bei BMW hinterlässt die Ukraine-Kris...,https://www.derstandard.at/story/2000134170212...,2022-03-16T18:47,Geopolitische Probleme,15,NaN,Ukraine Krieg kosten Autobauer Rendite,vor alle bei BMW hinterlässen der Ukraine-Kris...
9997,Flüchtlingsversorgung: Herausforderung annehmen,"Sollte es nicht zeitnah gelingen, die russisch...",https://www.derstandard.at/story/2000134170520...,2022-03-16T18:40,Irene Brickner,69,Kommentar,Flüchtlingsversorgung -- Herausforderung annehmen,sollen es nicht zeitnah gelingen -- der russis...
9998,Nach der Strabag geht auch Sigi Wolf auf Dista...,Die Kooperation von Steyr mit der russischen G...,https://www.derstandard.at/story/2000134170253...,2022-03-16T18:37,Kooperation,52,NaN,nach der Strabag gehen auch Sigi Wolf auf Dist...,der Kooperation von Steyr mit der russisch GAZ...


## Step 3: Embeddings

Two new columns are created. They contain lists of vectors. A vector represents embedding of a single token, a title then is converted to a list of such token. The numeric representation of text enables us standard training of a neural net with the vectors as inputs.

In [49]:
standard_data['title_vectors'] = standard_data['title_tokens']
standard_data['subtitle_vectors'] = standard_data['subtitle_tokens']
nlp.get_pipe("tok2vec")

for index, row in standard_data.iterrows():
    
    token_list = nlp(row['title_tokens'])
    vector_list = [token.vector for token in token_list]
    standard_data.at[index, 'title_vectors'] = vector_list 

    try:

        token_list = nlp(row['subtitle_tokens'])
        vector_list = [token.vector for token in token_list]

    except:
        token_list = []

    standard_data.at[index, 'subtitle_vectors'] = vector_list



In [58]:
standard_data

'USA lassen Pfizers Covid-Tablette Paxlovid für den Hausgebrauch zu'